# EDA + Data Aggregation (Housing Data)

In [67]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#from pandas_profiling import ProfileReport

## Data aggregations 
#### Merged set w unemployment rates + Freddie Mac interest rates

In [68]:
file_path = '../datasets/merged/rdc_data_merged_with_unemployment.csv'
df = pd.read_csv(file_path)

/Users/blake/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0,1,37,38) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [69]:
file_path_int5 = '../datasets/fed/weekly_arm_percent_of_total_MORTGAGE5US.csv'
int_5 = pd.read_csv(file_path_int5)
file_path_int15 = '../datasets/fed/weekly_15_yr_fixed_percent_of_total_MORTGAGE15US.csv'
int_15 = pd.read_csv(file_path_int15)
file_path_int30 = '../datasets/fed/historial_mortgate_rates_MORTGAGE30US.csv'
int_30 = pd.read_csv(file_path_int30)

In [9]:
df.head()

,month_date_yyyymm,county_fips,county_name,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,...,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy,quality_flag,unemployment_rate
0,202208,45001,"abbeville, sc",199000.0,-0.1152,-0.3671,32.0,0.0323,-0.0857,67.0,...,-0.1517,-0.3672,48.0,-0.0204,-0.2836,0.4375,-0.1431,-0.4768,1.0,NaN
1,202207,45001,"abbeville, sc",224900.0,0.1251,-0.1172,31.0,0.0333,0.0000,42.0,...,0.1388,-0.2150,49.0,-0.2097,-0.1695,0.5806,-0.4527,-0.3226,1.0,4.1
2,202206,45001,"abbeville, sc",199900.0,0.0060,-0.2858,30.0,0.1111,0.0714,37.0,...,0.0352,-0.3666,62.0,0.0508,0.1923,1.0333,-0.1889,0.2119,1.0,4.2
3,202205,45001,"abbeville, sc",198700.0,-0.0303,-0.4321,27.0,0.5882,1.0769,32.0,...,0.0007,-0.5366,59.0,0.2553,0.7353,1.2222,-0.5425,-0.3932,1.0,3.5
4,202204,45001,"abbeville, sc",204900.0,-0.0002,-0.0196,17.0,-0.1053,0.0000,33.0,...,-0.3195,-0.4679,47.0,0.0217,0.4688,1.7647,0.4489,0.8824,1.0,3.0


In [ ]:
df.shape

In [10]:
df.columns

Index(['month_date_yyyymm', 'county_fips', 'county_name',
       'median_listing_price', 'median_listing_price_mm',
       'median_listing_price_yy', 'active_listing_count',
       'active_listing_count_mm', 'active_listing_count_yy',
       'median_days_on_market', 'median_days_on_market_mm',
       'median_days_on_market_yy', 'new_listing_count', 'new_listing_count_mm',
       'new_listing_count_yy', 'price_increased_count',
       'price_increased_count_mm', 'price_increased_count_yy',
       'price_reduced_count', 'price_reduced_count_mm',
       'price_reduced_count_yy', 'pending_listing_count',
       'pending_listing_count_mm', 'pending_listing_count_yy',
       'median_listing_price_per_square_foot',
       'median_listing_price_per_square_foot_mm',
       'median_listing_price_per_square_foot_yy', 'median_square_feet',
       'median_square_feet_mm', 'median_square_feet_yy',
       'average_listing_price', 'average_listing_price_mm',
       'average_listing_price_yy', 'total_l

In [8]:
int_5.head()

,DATE,MORTGAGE5US
0,2005-01-06,5.03
1,2005-01-13,5.05
2,2005-01-20,5.05
3,2005-01-27,5.02
4,2005-02-03,5.00


In [70]:
# aggregate mortgage data by monthly average
int_5['month'] = int_5.DATE.str[:7]
grouped_int5 = int_5.groupby('month').mean()
int_15['month'] = int_15.DATE.str[:7]
grouped_int15 = int_15.groupby('month').mean()
int_30['month'] = int_30.DATE.str[:7]
grouped_int30 = int_30.groupby('month').mean()

In [71]:
#int_5_val = grouped_int_5['MORTGAGE5US'].where(grouped_int_5['month'] == comb)
#grouped_int30
# #grouped_int5['MORTGAGE5US'].where(grouped_int5.index == '2022-08') 
# grouped_int5.loc[grouped_int5.index == '2022-08', 'MORTGAGE5US'][0]

In [72]:
int5_ls = []
int15_ls = []
int30_ls = []
for i, v in enumerate(df.month_date_yyyymm):
    vs = str(v)
    yr = vs[:4]
    mn = vs[4:]
    comb = str(yr) + '-' + str(mn)
    #find the proper mortgage value or save value as "none" it it doesn't exist
    try :
        int_5_val = round(grouped_int5.loc[grouped_int5.index == comb, 'MORTGAGE5US'][0],4)
    except:
        int_5_val = 'none'
    try: 
        int_15_val = round(grouped_int15.loc[grouped_int15.index == comb, 'MORTGAGE15US'][0],4)
    except:
        int_15_val = 'none'
    try:
        int_30_val = round(grouped_int30.loc[grouped_int30.index == comb, 'MORTGAGE30US'][0],4)
    except: 
        int_30_val = 'none'
        
    int5_ls.append(int_5_val)
    int15_ls.append(int_15_val)
    int30_ls.append(int_30_val)
df['5yr_interest_rate'] = int5_ls
df['15yr_interest_rate'] = int15_ls
df['30yr_interest_rate'] = int30_ls

In [14]:
df.head()

,month_date_yyyymm,county_fips,county_name,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,...,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy,quality_flag,unemployment_rate,5yr_interest_rate,15yr_interest_rate,30yr_interest_rate
0,202208,45001,"abbeville, sc",199000.0,-0.1152,-0.3671,32.0,0.0323,-0.0857,67.0,...,-0.0204,-0.2836,0.4375,-0.1431,-0.4768,1.0,NaN,4.3575,4.5625,5.2225
1,202207,45001,"abbeville, sc",224900.0,0.1251,-0.1172,31.0,0.0333,0.0000,42.0,...,-0.2097,-0.1695,0.5806,-0.4527,-0.3226,1.0,4.1,4.285,4.6125,5.4125
2,202206,45001,"abbeville, sc",199900.0,0.0060,-0.2858,30.0,0.1111,0.0714,37.0,...,0.0508,0.1923,1.0333,-0.1889,0.2119,1.0,4.2,4.28,4.652,5.522
3,202205,45001,"abbeville, sc",198700.0,-0.0303,-0.4321,27.0,0.5882,1.0769,32.0,...,0.2553,0.7353,1.2222,-0.5425,-0.3932,1.0,3.5,4.055,4.435,5.23
4,202204,45001,"abbeville, sc",204900.0,-0.0002,-0.0196,17.0,-0.1053,0.0000,33.0,...,0.0217,0.4688,1.7647,0.4489,0.8824,1.0,3.0,3.695,4.215,4.9825


## EDA
#### Interest rate EDA and correlation analysis between key values

In [ ]:
# check the count of none values of interest rates
print(sum(df['5yr_interest_rate'] == 'none'))
print(sum(df['15yr_interest_rate'] == 'none'))
print(sum(df['30yr_interest_rate'] == 'none'))

In [ ]:
df.loc[df['30yr_interest_rate'] == 'none']

In [73]:
#dfc will be a cleaned data frame not including the one row that wasn't housing data
dfc = df.drop(labels=229214)

In [ ]:
print(sum(dfc['5yr_interest_rate'] == 'none'))
print(sum(dfc['15yr_interest_rate'] == 'none'))
print(sum(dfc['30yr_interest_rate'] == 'none'))

In [ ]:
# profile = ProfileReport(df, title="Pandas Profiling Report")

In [74]:
#dfc['month_date_yyyymm'] = pd.to_datetime(dfc['month_date_yyyymm'])
dfc['month_date_yyyymm'] = pd.to_datetime(dfc['month_date_yyyymm'], format='%Y%m')
dfc_g = dfc.sort_values(['month_date_yyyymm'])

In [ ]:
# let's take a look at the 3 interest rates over time to start
plt.plot(dfc_g.month_date_yyyymm, dfc_g['5yr_interest_rate'], label = "5yr interest")
plt.plot(dfc_g.month_date_yyyymm, dfc_g['15yr_interest_rate'], label = "15yr interest")
plt.plot(dfc_g.month_date_yyyymm, dfc_g['30yr_interest_rate'], label = "30yr interest")
plt.legend()
plt.show()

5, 15, and 30 year interest rates overall have similar trends with the 30 yr interest rate trending higher than the others almost always. Interest rates became drastically low during covid and continued to decrease until the end of 2021. Since 2022 began rates have risen sharply to all time highs since 2017.

In [75]:
# Since interest rates are national, let's build a table to view average prices and average unemployment
nd = dfc.groupby('month_date_yyyymm').mean()
nd.reset_index(inplace=True)
nd = nd.rename(columns = {'index':'month_date_yyyymm'})
nd.head()
#pull correct monthy int rates
# grouped_int30 = int_30.groupby('month').mean()
# grouped_int30.head()


,month_date_yyyymm,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,median_days_on_market_mm,median_days_on_market_yy,...,median_square_feet_yy,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,quality_flag,unemployment_rate
0,2016-07-01,200407.688189,NaN,NaN,502.671966,NaN,NaN,95.794614,NaN,NaN,...,NaN,271046.352044,NaN,NaN,638.539585,NaN,NaN,0.196891,NaN,5.013173
1,2016-08-01,195822.020104,NaN,NaN,499.866083,NaN,NaN,98.528534,NaN,NaN,...,NaN,264305.782750,NaN,NaN,628.564202,NaN,NaN,inf,NaN,4.811089
2,2016-09-01,196052.288031,NaN,NaN,494.903990,NaN,NaN,102.268570,NaN,NaN,...,NaN,264719.563737,NaN,NaN,616.115148,NaN,NaN,inf,NaN,4.553681
3,2016-10-01,195181.369001,NaN,NaN,481.159857,NaN,NaN,105.590467,NaN,NaN,...,NaN,265205.207847,NaN,NaN,596.772374,NaN,NaN,0.177529,NaN,4.452205
4,2016-11-01,195508.568966,NaN,NaN,459.274886,NaN,NaN,110.537736,NaN,NaN,...,NaN,266512.741379,NaN,NaN,568.332466,NaN,NaN,inf,NaN,4.341705


In [76]:
# grouped_int5.loc[grouped_int5.index == comb, 'MORTGAGE5US'][0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
int5_ls_g = []
int15_ls_g = []
int30_ls_g = []
for i, v in enumerate(nd.month_date_yyyymm):
    vs = str(v)
    yr = vs[:4]
    mn = vs[5:7]
    comb = str(yr) + '-' + str(mn) 
    #find the proper mortgage value or save value as "none" it it doesn't exist
    try :
        int_5_val = round(grouped_int5.loc[grouped_int5.index == comb, 'MORTGAGE5US'][0],4)
    except:
        int_5_val = 'none'
    try: 
        int_15_val = round(grouped_int15.loc[grouped_int15.index == comb, 'MORTGAGE15US'][0],4)
    except:
        int_15_val = 'none'
    try:
        int_30_val = round(grouped_int30.loc[grouped_int30.index == comb, 'MORTGAGE30US'][0],4)
    except: 
        int_30_val = 'none'
        
    int5_ls_g.append(int_5_val)
    int15_ls_g.append(int_15_val)
    int30_ls_g.append(int_30_val)
nd['5yr_interest_rate'] = int5_ls_g
nd['15yr_interest_rate'] = int15_ls_g
nd['30yr_interest_rate'] = int30_ls_g

In [20]:
nd.head()

,month_date_yyyymm,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,median_days_on_market_mm,median_days_on_market_yy,...,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,quality_flag,unemployment_rate,5yr_interest_rate,15yr_interest_rate,30yr_interest_rate
0,2016-07-01,200407.688189,NaN,NaN,502.671966,NaN,NaN,95.794614,NaN,NaN,...,NaN,638.539585,NaN,NaN,0.196891,NaN,5.383484,2.7500,2.7475,3.440
1,2016-08-01,195822.020104,NaN,NaN,499.866083,NaN,NaN,98.528534,NaN,NaN,...,NaN,628.564202,NaN,NaN,inf,NaN,5.168025,2.7450,2.7450,3.435
2,2016-09-01,196052.288031,NaN,NaN,494.903990,NaN,NaN,102.268570,NaN,NaN,...,NaN,616.115148,NaN,NaN,inf,NaN,4.893343,2.8140,2.7560,3.460
3,2016-10-01,195181.369001,NaN,NaN,481.159857,NaN,NaN,105.590467,NaN,NaN,...,NaN,596.772374,NaN,NaN,0.177529,NaN,4.784181,2.8275,2.7625,3.470
4,2016-11-01,195508.568966,NaN,NaN,459.274886,NaN,NaN,110.537736,NaN,NaN,...,NaN,568.332466,NaN,NaN,inf,NaN,4.664942,2.9850,3.0275,3.770


In [ ]:
# now let's look at average unemployment nationally vs interest rates
plt.plot(nd.month_date_yyyymm, nd['unemployment_rate'], label = "unemployment rate")
plt.plot(nd.month_date_yyyymm, nd['5yr_interest_rate'], label = "5yr interest")
plt.plot(nd.month_date_yyyymm, nd['15yr_interest_rate'], label = "15yr interest")
plt.plot(nd.month_date_yyyymm, nd['30yr_interest_rate'], label = "30yr interest")
plt.legend()
plt.show()

The interest rates during covid, from 2020 to the end of 2021, took a dip, which correlates with the high unemployment rates during that time period. The two data points however don't seem to be incredibly closely correlated, pointing to additional factors affecting interest rates besides just unemployment rates.

In [ ]:
# average housing price vs interest rates
ax1 = plt.subplot()
l1, = ax1.plot(nd.month_date_yyyymm, nd['median_listing_price'], label = "avg listing price", color = 'purple')
ax2 = ax1.twinx()
l2, = ax2.plot(nd.month_date_yyyymm, nd['5yr_interest_rate'], label = "5yr interest")
l3, = ax2.plot(nd.month_date_yyyymm, nd['15yr_interest_rate'], label = "15yr interest")
l4, = ax2.plot(nd.month_date_yyyymm, nd['30yr_interest_rate'], label = "30yr interest")
ax1.set_ylabel('avg price')
ax2.set_ylabel('int rate')
plt.legend([l1, l2, l3, l4], ["avg listing price", "5yr interest", '15yr interest', '30yr interest'])
plt.show()

The average housing prices since 2017 have been gradually increasing, while the interest rates of home loans took a major dip during covid. The two data points don't seem to be highly correlated.

## Aggregate GDP data

In [ ]:
file_path_GDP = '../datasets/bea/GDP_State_Quarter.csv'
gdp = pd.read_csv(file_path_GDP)

In [23]:
gdp.head()

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2005:Q1,2005:Q2,...,2019:Q4,2020:Q1,2020:Q2,2020:Q3,2020:Q4,2021:Q1,2021:Q2,2021:Q3,2021:Q4,2022:Q1
0,"""00000""",United States,,SQGDP1,1.0,...,Real GDP (millions of chained 2012 dollars),Millions of chained 2012 dollars,1.476785e+07,1.483971e+07,...,1.920231e+07,1.895199e+07,1.725820e+07,1.856077e+07,1.876778e+07,1.905566e+07,1.936831e+07,1.947889e+07,1.980629e+07,1.972792e+07
1,"""00000""",United States,,SQGDP1,2.0,...,Chain-type quantity indexes for real GDP,Quantity index,9.085700e+01,9.129900e+01,...,1.181390e+02,1.165990e+02,1.061780e+02,1.141920e+02,1.154660e+02,1.172370e+02,1.191600e+02,1.198410e+02,1.218550e+02,1.213730e+02
2,"""00000""",United States,,SQGDP1,3.0,...,Current-dollar GDP (millions of current dollars),Millions of current dollars,1.276729e+07,1.292266e+07,...,2.169446e+07,2.148137e+07,1.947744e+07,2.113857e+07,2.147760e+07,2.203823e+07,2.274096e+07,2.320234e+07,2.400282e+07,2.438673e+07
3,"""01000""",Alabama,5,SQGDP1,1.0,...,Real GDP (millions of chained 2012 dollars),Millions of chained 2012 dollars,1.826004e+05,1.846257e+05,...,2.043541e+05,2.020691e+05,1.838903e+05,2.003543e+05,2.013106e+05,2.025988e+05,2.041358e+05,2.049688e+05,2.078352e+05,2.068797e+05
4,"""01000""",Alabama,5,SQGDP1,2.0,...,Chain-type quantity indexes for real GDP,Quantity index,9.648900e+01,9.755900e+01,...,1.079840e+02,1.067760e+02,9.717000e+01,1.058700e+02,1.063750e+02,1.070560e+02,1.078680e+02,1.083080e+02,1.098230e+02,1.093180e+02


In [ ]:
# pull all instances where line code = 1
gdp = gdp.loc[gdp['LineCode'] == 1]

In [ ]:
gdp.head()

In [ ]:
# total_rows['ColumnID'] = total_rows['ColumnID'].astype(str)

In [46]:
full_state = abbrev_to_state['me']
gdp_val_s = gdp.loc[gdp.GeoName == full_state, comb][0]
gdp_val = gdp_val_s.values[0]
gdp_val = gdp.loc[gdp.GeoName == full_state, '2021:Q1']
print(gpd_val.values[0])

60933.5


In [77]:
Quarterly_GDP_State_Chained_2012 = []

for i, v in enumerate(dfc.month_date_yyyymm):
    vs = str(v)
    yr = vs[:4]
    mn = vs[4:]
    mnn = float(mn[1:3])
    qrt = int((mnn-1)//3 + 1)
    comb = str(yr) + ':Q' + str(qrt) # get proper YYYY:Q# value for GDP dataset
    # get proper State value 
    location = dfc.iloc[i,2]
    state = location[-2:]
    abbrev_to_state = {
    "al":"Alabama",
    "ak": "Alaska",
    "az":"Arizona",
    'ar':"Arkansas",
    'ca' : "California",
    'co' : "Colorado",
    'ct' : "Connecticut",
    'de' : "Delaware",
    'fl' : "Florida",
    'ga' : "Georgia",
    'hi': "Hawaii",
    'id':"Idaho",
    'il':"Illinois",
    'in':"Indiana",
    'ia':"Iowa",
    'ks' : "Kansas",
    'ky': "Kentucky",
    'la' : "Louisiana",
    'me' : "Maine",
    'md' : "Maryland",
    'ma' : "Massachusetts",
    'mi' : "Michigan",
    'mn' : "Minnesota",
    'ms' : "Mississippi",
    'mo':"Missouri",
    'mt' : "Montana",
    'ne' : "Nebraska",
    'nv' :"Nevada",
    'nh':"New Hampshire",
    'nj' : "New Jersey",
    'nm' : "New Mexico",
    'ny' : "New York",
    'nc' : "North Carolina",
    'nd' : "North Dakota",
    'oh' : "Ohio",
    'ok':"Oklahoma",
    'or':"Oregon",
    'pa':"Pennsylvania",
    'ri':"Rhode Island",
    'sc' : "South Carolina",
    'sd' : "South Dakota",
    'tn' : "Tennessee",
    'tx' : "Texas",
    'ut' : "Utah",
    'vt' : "Vermont",
    'va' : "Virginia",
    'wa' : "Washington",
    'wv' : "West Virginia",
    'wi':"Wisconsin",
    'wy' : "Wyoming",
    'dc' : "District of Columbia",
    'as':"American Samoa",
    'gu':"Guam",
    'mp':"Northern Mariana Islands",
    'pr' : "Puerto Rico",
    'um':"United States Minor Outlying Islands",
    'vi' : "U.S. Virgin Islands",
    }
    try:
        full_state = abbrev_to_state[state]
    except:
        full_state = 'none'
    #find the proper GDP value or save value as "none" it it doesn't exist
    try :
        gdp_val_s = gdp.loc[gdp.GeoName == full_state, comb]
        gdp_val = gdp_val_s.values[0]
#         print(gdp_val)
    except:
        gdp_val = 'none'
        
    Quarterly_GDP_State_Chained_2012.append(gdp_val)
dfc['Quarterly_GDP_State_Chained_2012'] = Quarterly_GDP_State_Chained_2012

In [78]:
dfc.head()

,month_date_yyyymm,county_fips,county_name,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,...,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy,quality_flag,unemployment_rate,5yr_interest_rate,15yr_interest_rate,30yr_interest_rate,Quarterly_GDP_State_Chained_2012
0,2022-08-01,45001,"abbeville, sc",199000.0,-0.1152,-0.3671,32.0,0.0323,-0.0857,67.0,...,-0.2836,0.4375,-0.1431,-0.4768,1.0,0.0,4.3575,4.5625,5.2225,none
1,2022-07-01,45001,"abbeville, sc",224900.0,0.1251,-0.1172,31.0,0.0333,0.0000,42.0,...,-0.1695,0.5806,-0.4527,-0.3226,1.0,4.1,4.285,4.6125,5.4125,none
2,2022-06-01,45001,"abbeville, sc",199900.0,0.0060,-0.2858,30.0,0.1111,0.0714,37.0,...,0.1923,1.0333,-0.1889,0.2119,1.0,4.2,4.28,4.652,5.522,none
3,2022-05-01,45001,"abbeville, sc",198700.0,-0.0303,-0.4321,27.0,0.5882,1.0769,32.0,...,0.7353,1.2222,-0.5425,-0.3932,1.0,3.5,4.055,4.435,5.23,none
4,2022-04-01,45001,"abbeville, sc",204900.0,-0.0002,-0.0196,17.0,-0.1053,0.0000,33.0,...,0.4688,1.7647,0.4489,0.8824,1.0,3.0,3.695,4.215,4.9825,none


In [79]:
dfc.loc[dfc.month_date_yyyymm =='2021-01-01','Quarterly_GDP_State_Chained_2012']

19             217344
93             224326
167            485850
241           77265.9
315            178622
             ...     
228851        49820.2
228925         331004
228999         357383
229073    1.78652e+06
229147    1.78652e+06
Name: Quarterly_GDP_State_Chained_2012, Length: 3104, dtype: object

In [82]:
print(sum(dfc['Quarterly_GDP_State_Chained_2012'] == 'none'))
print((dfc.shape[0]-sum(dfc['Quarterly_GDP_State_Chained_2012'] == 'none'))/dfc.shape[0])

15500
0.9323776034622667


93% of rows have data for Quarterly GDP by State (chained to 2012)

## Aggregate county-level, annual personal income data

In [83]:
# read in two personal income datasets
file_path_PI_county = '../datasets/bea/County_Personal_Income_Yearly.csv'
pic = pd.read_csv(file_path_PI_county)
file_path_PI_state = '../datasets/bea/State_Personal_Income_Quarterly.csv'
pis = pd.read_csv(file_path_PI_state)

/Users/blake/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [84]:
pic[0:15]

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,1969,1970,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,"""00000""",United States,,CAINC4,10.0,...,Personal income (thousands of dollars),Thousands of dollars,791229000,855525000,...,13330436000,14003346000,14189228000,14969527000,15681233000,16092713000,16845028000,17681159000,18402004000,19607447000
1,"""00000""",United States,,CAINC4,11.0,...,Nonfarm personal income 1/,Thousands of dollars,773954000,838026000,...,13238101000,13912005000,14065574000,14867407000,15592764000,16023802000,16770381000,17618281000,18335714000,19502393000
2,"""00000""",United States,,CAINC4,12.0,...,Farm income 2/,Thousands of dollars,17275000,17499000,...,92335000,91341000,123654000,102120000,88469000,68911000,74647000,62878000,66290000,105054000
3,"""00000""",United States,,CAINC4,20.0,...,Population (persons) 3/,Number of persons,201298000,203798722,...,311583481,313877662,316059947,318386329,320738994,323071755,325122128,326838199,328329953,329484123
4,"""00000""",United States,,CAINC4,30.0,...,Per capita personal income (dollars) 4/,Dollars,3931,4198,...,42783,44614,44894,47017,48891,49812,51811,54098,56047,59510
5,"""00000""",United States,,CAINC4,35.0,...,Earnings by place of work,Thousands of dollars,653665000,693352000,...,9444365000,9902321000,10232098000,10688401000,11115543000,11384770000,11926173000,12515233000,13024685000,13202475000
6,"""00000""",United States,,CAINC4,36.0,...,Less: Contributions for government social insu...,Thousands of dollars,43791000,46012000,...,915062000,948953000,1102460000,1151698000,1202909000,1237020000,1296965000,1358033000,1418846000,1456843000
7,"""00000""",United States,,CAINC4,37.0,...,Employee and self-employed contributions for ...,Thousands of dollars,21204000,22345000,...,423231000,436562000,577062000,606203000,634466000,656982000,692924000,734505000,770529000,794461000
8,"""00000""",United States,,CAINC4,38.0,...,Employer contributions for government social ...,Thousands of dollars,22587000,23667000,...,491831000,512391000,525398000,545495000,568443000,580038000,604041000,623528000,648317000,662382000
9,"""00000""",United States,,CAINC4,42.0,...,Plus: Adjustment for residence 6/,Thousands of dollars,-107000,-112000,...,3201000,3411000,3651000,3438000,3403000,3062000,2980000,3451000,3553000,3697000


In [111]:
pic['LineCode'] = pic['LineCode'].astype(str)

In [93]:
# 10, 20, 30, and 7010 linecodes seem like valuable attributes
# make separate data sets for each linecode
pic_PI = pic.loc[pic['LineCode'] == '10.0']
pic_pop = pic.loc[pic['LineCode'] == '20.0']
pic_pcpop = pic.loc[pic['LineCode'] == '30.0']
pic_totemp = pic.loc[pic['LineCode'] == '7010.0']

In [94]:
pic_totemp.head()

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,1969,1970,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
20,"""00000""",United States,,CAINC4,7010.0,...,Total employment,Number of jobs,91053200,91277600,...,176091700,178979700,182325100,186233800,190325800,193425900,196393100,200280200,201644200,190776800
43,"""01000""",Alabama,5,CAINC4,7010.0,...,Total employment,Number of jobs,1411234,1412928,...,2497933,2503678,2523338,2551872,2587451,2619737,2649812,2692383,2712704,2612469
66,"""01001""","Autauga, AL",5,CAINC4,7010.0,...,Total employment,Number of jobs,6630,6853,...,17298,16421,16476,16858,17278,17435,17801,18165,18307,17802
89,"""01003""","Baldwin, AL",5,CAINC4,7010.0,...,Total employment,Number of jobs,20008,19749,...,92000,93226,96469,99807,103917,107829,111661,115716,117031,112817
112,"""01005""","Barbour, AL",5,CAINC4,7010.0,...,Total employment,Number of jobs,9428,9448,...,12359,12254,12153,11935,12053,11587,11635,11504,11827,11435


In [130]:
# aggregate personal income data by county (annual)
personal_income_county_annual = []

for i, v in enumerate(dfc.month_date_yyyymm):
    vs = str(v)
    yr = vs[:4]
    # get proper FIPS value 
    location = str(dfc.iloc[i,1])
    if len(location) == 4:
        location = '\"0'+location + '\"'
    elif len(location) == 3:
        location = '\"00' + location + '\"'
    elif len(location) == 2:
        location = '\"000' + location + '\"'
    elif len(location) == 1:
        location = '\"0000' + location + '\"'
    else:
        location = '\"' + location + '\"'
    #find the proper Personal income value or save value as "none" it it doesn't exist
    try :
        pi_val_s = pic_PI.loc[pic_PI.GeoFIPS == location, yr]
        pi_val = pi_val_s.values[0]
    except:
        pi_val = 'none'
        
    personal_income_county_annual.append(pi_val)
dfc['personal_income_county_annual'] = personal_income_county_annual

"45001"
"45001"
"45001"
"45001"
"45001"
"45001"
"45001"
"45001"
"45001"
"45001"
"45001"
"45001"
"45001"
"45001"
"45001"
"45001"
"45001"
"45001"
"45001"
"45001"
"45001"
928161
"45001"
928161
"45001"
928161
"45001"
928161
"45001"
928161
"45001"
928161
"45001"
928161
"45001"
928161
"45001"
928161
"45001"
928161


In [125]:
# pic_PI.loc[pic_PI.GeoFIPS == '\"01003\"', '2019']

#         gdp_val_s = gdp.loc[gdp.GeoName == full_state, comb]
#         gdp_val = gdp_val_s.values[0]

In [132]:
# dfc.loc[dfc.county_fips ==45001,'personal_income_county_annual']
print(sum(dfc['personal_income_county_annual'] == 'none'))
print((dfc.shape[0]-sum(dfc['personal_income_county_annual'] == 'none'))/dfc.shape[0])

64574
0.7182807332885426


71.% of rows have the data attribute personal income by county annually

In [133]:
# aggregate population data by county (annual)
population_county_annual = []
for i, v in enumerate(dfc.month_date_yyyymm):
    vs = str(v)
    yr = vs[:4]
    # get proper FIPS value 
    location = str(dfc.iloc[i,1])
    if len(location) == 4:
        location = '\"0'+location + '\"'
    elif len(location) == 3:
        location = '\"00' + location + '\"'
    elif len(location) == 2:
        location = '\"000' + location + '\"'
    elif len(location) == 1:
        location = '\"0000' + location + '\"'
    else:
        location = '\"' + location + '\"'
    #find the proper Personal income value or save value as "none" it it doesn't exist
    try :
        pop_val_s = pic_pop.loc[pic_pop.GeoFIPS == location, yr]
        pop_val = pop_val_s.values[0]
    except:
        pop_val = 'none'
        
    population_county_annual.append(pop_val)
dfc['population_county_annual'] = population_county_annual

In [134]:
print(sum(dfc['population_county_annual'] == 'none'))
print((dfc.shape[0]-sum(dfc['population_county_annual'] == 'none'))/dfc.shape[0])

64574
0.7182807332885426


71.8% of rows have a new data attribute county population, measured annually

In [136]:
# aggregate personal income per capital data by county (annual)
population_per_capita_county_annual = []
for i, v in enumerate(dfc.month_date_yyyymm):
    vs = str(v)
    yr = vs[:4]
    # get proper FIPS value 
    location = str(dfc.iloc[i,1])
    if len(location) == 4:
        location = '\"0'+location + '\"'
    elif len(location) == 3:
        location = '\"00' + location + '\"'
    elif len(location) == 2:
        location = '\"000' + location + '\"'
    elif len(location) == 1:
        location = '\"0000' + location + '\"'
    else:
        location = '\"' + location + '\"'
    #find the proper Personal income value or save value as "none" it it doesn't exist
    try :
        pcpop_val_s = pic_pcpop.loc[pic_pcpop.GeoFIPS == location, yr]
        pcpop_val = pcpop_val_s.values[0]
    except:
        pcpop_val = 'none'
        
    population_per_capita_county_annual.append(pcpop_val)
dfc['population_per_capita_county_annual'] = population_per_capita_county_annual

In [137]:
print(sum(dfc['population_per_capita_county_annual'] == 'none'))
print((dfc.shape[0]-sum(dfc['population_per_capita_county_annual'] == 'none'))/dfc.shape[0])

64574
0.7182807332885426


71.8% of rows have new data attribute, population per capita by county, annually

In [139]:
# aggregate employment data by county (annual)
employed_county_annual = []
for i, v in enumerate(dfc.month_date_yyyymm):
    vs = str(v)
    yr = vs[:4]
    # get proper FIPS value 
    location = str(dfc.iloc[i,1])
    if len(location) == 4:
        location = '\"0'+location + '\"'
    elif len(location) == 3:
        location = '\"00' + location + '\"'
    elif len(location) == 2:
        location = '\"000' + location + '\"'
    elif len(location) == 1:
        location = '\"0000' + location + '\"'
    else:
        location = '\"' + location + '\"'
    #find the proper Personal income value or save value as "none" it it doesn't exist
    try :
        emp_val_s = pic_totemp.loc[pic_totemp.GeoFIPS == location, yr]
        emp_val = emp_val_s.values[0]
    except:
        emp_val = 'none'
        
    employed_county_annual.append(emp_val)
dfc['employed_county_annual'] = employed_county_annual

In [140]:
print(sum(dfc['employed_county_annual'] == 'none'))
print((dfc.shape[0]-sum(dfc['employed_county_annual'] == 'none'))/dfc.shape[0])

64574
0.7182807332885426


71.8% of rows have new data attribute # employed per county, annually

## Aggregate quarterly state-level personal income data

In [63]:
pis.head()

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,1948:Q1,1948:Q2,...,2019:Q4,2020:Q1,2020:Q2,2020:Q3,2020:Q4,2021:Q1,2021:Q2,2021:Q3,2021:Q4,2022:Q1
0,"""00000""",United States,,SQINC1,1.0,...,"Personal income (millions of dollars, seasonal...",Millions of dollars,204641.7,210069.4,...,18623043.4,18822075.9,20329653.1,19755199.7,19522859.3,21849043.1,20647726.9,20803215.4,20989733.9,21237207.4
1,"""00000""",United States,,SQINC1,2.0,...,"Population (midperiod, persons) 1/",Number of persons,(NA),(NA),...,330971591.0,331295939.0,331448217.0,331596557.0,331734262.0,331706294.0,331776226.0,332049982.0,332336782.0,332502197.0
2,"""00000""",United States,,SQINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,(NA),(NA),...,56268.0,56813.0,61336.0,59576.0,58851.0,65869.0,62234.0,62651.0,63158.0,63871.0
3,"""01000""",Alabama,5,SQINC1,1.0,...,"Personal income (millions of dollars, seasonal...",Millions of dollars,2496.0,2595.6,...,218325.6,220503.8,239992.0,228757.7,225741.7,260454.2,237896.6,239116.2,243278.2,245380.1
4,"""01000""",Alabama,5,SQINC1,2.0,...,"Population (midperiod, persons) 1/",Number of persons,(NA),(NA),...,5015982.0,5021627.0,5024115.0,5027375.0,5031760.0,5033508.0,5036858.0,5043548.0,5050555.0,5055254.0


In [141]:
# pull only per capita personal income data
pis['LineCode'] = pis['LineCode'].astype(str)
pis_PI = pis.loc[pis['LineCode'] == '3.0']

In [142]:
pis_PI.head()

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,1948:Q1,1948:Q2,...,2019:Q4,2020:Q1,2020:Q2,2020:Q3,2020:Q4,2021:Q1,2021:Q2,2021:Q3,2021:Q4,2022:Q1
2,"""00000""",United States,,SQINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,(NA),(NA),...,56268.0,56813.0,61336.0,59576.0,58851.0,65869.0,62234.0,62651.0,63158.0,63871.0
5,"""01000""",Alabama,5,SQINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,(NA),(NA),...,43526.0,43911.0,47768.0,45502.0,44863.0,51744.0,47231.0,47410.0,48169.0,48540.0
8,"""02000""",Alaska *,8,SQINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,(NA),(NA),...,62135.0,61899.0,66492.0,62293.0,62708.0,69440.0,65855.0,66115.0,67420.0,67742.0
11,"""04000""",Arizona,6,SQINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,(NA),(NA),...,47847.0,48413.0,53994.0,52153.0,50759.0,56991.0,53007.0,53409.0,54290.0,54422.0
14,"""05000""",Arkansas,5,SQINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,(NA),(NA),...,45455.0,45776.0,50314.0,47019.0,46888.0,53989.0,49891.0,50088.0,50335.0,50943.0


In [147]:
# pull quarterly per capita personal income by state
personal_income_per_capita_state_quarterly = []
for i, v in enumerate(dfc.month_date_yyyymm):
    vs = str(v)
    yr = vs[:4]
    mn = vs[4:]
    mnn = float(mn[1:3])
    qrt = int((mnn-1)//3 + 1)
    comb = str(yr) + ':Q' + str(qrt) # get proper YYYY:Q# value for GDP dataset
    # get proper FIPS value 
    location = dfc.iloc[i,2]
    state = location[-2:]
    abbrev_to_state = {
    "al":"Alabama",
    "ak": "Alaska *",
    "az":"Arizona",
    'ar':"Arkansas",
    'ca' : "California",
    'co' : "Colorado",
    'ct' : "Connecticut",
    'de' : "Delaware",
    'fl' : "Florida",
    'ga' : "Georgia",
    'hi': "Hawaii",
    'id':"Idaho",
    'il':"Illinois",
    'in':"Indiana",
    'ia':"Iowa",
    'ks' : "Kansas",
    'ky': "Kentucky",
    'la' : "Louisiana",
    'me' : "Maine",
    'md' : "Maryland",
    'ma' : "Massachusetts",
    'mi' : "Michigan",
    'mn' : "Minnesota",
    'ms' : "Mississippi",
    'mo':"Missouri",
    'mt' : "Montana",
    'ne' : "Nebraska",
    'nv' :"Nevada",
    'nh':"New Hampshire",
    'nj' : "New Jersey",
    'nm' : "New Mexico",
    'ny' : "New York",
    'nc' : "North Carolina",
    'nd' : "North Dakota",
    'oh' : "Ohio",
    'ok':"Oklahoma",
    'or':"Oregon",
    'pa':"Pennsylvania",
    'ri':"Rhode Island",
    'sc' : "South Carolina",
    'sd' : "South Dakota",
    'tn' : "Tennessee",
    'tx' : "Texas",
    'ut' : "Utah",
    'vt' : "Vermont",
    'va' : "Virginia",
    'wa' : "Washington",
    'wv' : "West Virginia",
    'wi':"Wisconsin",
    'wy' : "Wyoming",
    'dc' : "District of Columbia",
    'as':"American Samoa",
    'gu':"Guam",
    'mp':"Northern Mariana Islands",
    'pr' : "Puerto Rico",
    'um':"United States Minor Outlying Islands",
    'vi' : "U.S. Virgin Islands",
    }
    try:
        full_state = abbrev_to_state[state]
    except:
        full_state = 'none'
    #find the proper Personal income per capita value or save value as "none" it it doesn't exist
    try :
        pcpi_val_s = pis_PI.loc[pis_PI.GeoName == full_state, comb]
        pcpi_val = pcpi_val_s.values[0]
    except:
        pcpi_val = 'none'
    personal_income_per_capita_state_quarterly.append(pcpi_val)
    
dfc['personal_income_per_capita_state_quarterly'] = personal_income_per_capita_state_quarterly

In [148]:
print(sum(dfc['personal_income_per_capita_state_quarterly'] == 'none'))
print((dfc.shape[0]-sum(dfc['personal_income_per_capita_state_quarterly'] == 'none'))/dfc.shape[0])

15776
0.931173488530369


93.1% of rows have a new data attribute: Personal income per capita, by state, quarterly

In [149]:
dfc.head()

,month_date_yyyymm,county_fips,county_name,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,...,unemployment_rate,5yr_interest_rate,15yr_interest_rate,30yr_interest_rate,Quarterly_GDP_State_Chained_2012,personal_income_county_annual,population_county_annual,population_per_capita_county_annual,employed_county_annual,personal_income_per_capita_state_quarterly
0,2022-08-01,45001,"abbeville, sc",199000.0,-0.1152,-0.3671,32.0,0.0323,-0.0857,67.0,...,0.0,4.3575,4.5625,5.2225,none,none,none,none,none,none
1,2022-07-01,45001,"abbeville, sc",224900.0,0.1251,-0.1172,31.0,0.0333,0.0000,42.0,...,4.1,4.285,4.6125,5.4125,none,none,none,none,none,none
2,2022-06-01,45001,"abbeville, sc",199900.0,0.0060,-0.2858,30.0,0.1111,0.0714,37.0,...,4.2,4.28,4.652,5.522,none,none,none,none,none,none
3,2022-05-01,45001,"abbeville, sc",198700.0,-0.0303,-0.4321,27.0,0.5882,1.0769,32.0,...,3.5,4.055,4.435,5.23,none,none,none,none,none,none
4,2022-04-01,45001,"abbeville, sc",204900.0,-0.0002,-0.0196,17.0,-0.1053,0.0000,33.0,...,3.0,3.695,4.215,4.9825,none,none,none,none,none,none


In [150]:
# now lets export our aggregated dataset to a csv
to_csv_path = '../datasets/merged/rdc_data_w_macroecon.csv'
dfc.to_csv(to_csv_path)